In [1]:
!pip install rank_bm25


In [2]:
from rank_bm25 import BM25Okapi
import pandas as pd
import nltk.corpus
import nltk
import nltk.stem
import rich
import re
import helper
import importlib

In [3]:
importlib.reload(helper)

<module 'helper' from '/notebooks/llm_search/notebooks/helper.py'>

In [4]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [5]:
def my_awesome_tokenizer(content, min_token_length=2):
    content = content.lower()
    
    # only keep alphanumeric
    content = re.sub(r'\W+', ' ', content)
    
    
    tokens = nltk.word_tokenize(content)
    lemmatizer = nltk.stem.WordNetLemmatizer()
    
    tokens_final = []
    
    for token in tokens:
        if  (   ( len(token) >= min_token_length )  and 
                ( token not in  nltk.corpus.stopwords.words("english")  )  
            ):
            
            token =lemmatizer.lemmatize(token)
            
            tokens_final.append(token)
            
            
    return tokens_final

In [6]:
def find_results(query:str ,df_corpus:pd.DataFrame, k =20):
    
    tokenized_query = my_awesome_tokenizer(query)
    doc_scores = bm25.get_scores(tokenized_query)
    
    tokens_in_vocab = [ token for token in tokenized_query  if token in bm25.idf]
    top_items = doc_scores.argsort()[-k:][::-1]
    
    degug_info = {
         "query_original":  query 
         , "tokenized_query":  tokenized_query 
         , "tokens_in_vocab" : tokens_in_vocab
        , "doc_scores":  doc_scores 
        , "top_items":  top_items 
    }
    rich.print (degug_info )
    
    df_subset = df_corpus.iloc[top_items]
    
    
    
    rich.print ( list (df_subset['product_title'] ) )
    
    helper.display_results(query,df_subset)



In [7]:
df = pd.read_parquet("../data/cleaned_input.parquet")

In [8]:
df_product = df[['product_id','product_title','url_product','url_image']].drop_duplicates(['product_id']).reset_index(drop=True)
df_product

,product_id,product_title,url_product,url_image
0,B000MOO21W,Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceil...,https://www.amazon.com/dp/B000MOO21W,http://images.amazon.com/images/P/B000MOO21W.0...
1,B07X3Y6B1V,Homewerks 7141-80 Bathroom Fan Integrated LED ...,https://www.amazon.com/dp/B07X3Y6B1V,http://images.amazon.com/images/P/B07X3Y6B1V.0...
2,B07WDM7MQQ,Homewerks 7140-80 Bathroom Fan Ceiling Mount E...,https://www.amazon.com/dp/B07WDM7MQQ,http://images.amazon.com/images/P/B07WDM7MQQ.0...
3,B07RH6Z8KW,Delta Electronics RAD80L BreezRadiance 80 CFM ...,https://www.amazon.com/dp/B07RH6Z8KW,http://images.amazon.com/images/P/B07RH6Z8KW.0...
4,B07QJ7WYFQ,Panasonic FV-08VRE2 Ventilation Fan with Reces...,https://www.amazon.com/dp/B07QJ7WYFQ,http://images.amazon.com/images/P/B07QJ7WYFQ.0...
...,...,...,...,...
1207284,0323377092,Neurology Self-Assessment: A Companion to Brad...,https://www.amazon.com/dp/0323377092,http://images.amazon.com/images/P/0323377092.0...
1207285,0323287832,"Bradley's Neurology in Clinical Practice, 2-Vo...",https://www.amazon.com/dp/0323287832,http://images.amazon.com/images/P/0323287832.0...
1207286,0198566344,Autonomic Failure: A Textbook of Clinical Diso...,https://www.amazon.com/dp/0198566344,http://images.amazon.com/images/P/0198566344.0...
1207287,0071845836,The Hospital Neurology Book,https://www.amazon.com/dp/0071845836,http://images.amazon.com/images/P/0071845836.0...


In [11]:
corpus = df_product['product_title']

In [12]:
bm25 = BM25Okapi(corpus=corpus, tokenizer=my_awesome_tokenizer)


In [13]:
query = "adidas shoes men 15"

In [14]:
find_results( query, df_product)


{
    'query_original': 'adidas shoes men 15',
    'tokenized_query': ['adidas', 'shoe', 'men', '15'],
    'tokens_in_vocab': ['adidas', 'shoe', 'men', '15'],
    'doc_scores': array([0., 0., 0., ..., 0., 0., 0.]),
    'top_items': array([ 554710,  551018,  551016,  973910,  551869, 1036411,  551007,
       1036410,  551680,  404428,  494213,  551630,  794753,  735241,
        546841,  555046,  546825,  554708,  720692,  551011])
}

[
    "adidas Men's Ultraboost DNA Running Shoe, Black/Black/Gold, 15",
    "adidas Men's Hoops 2.0 Mid Basketball Shoe, White/Royal/Scarlet, 15",
    "adidas Men's Duramo SL Running Shoe, Ink/Black/Indigo, 15",
    "adidas Dame 4 NBA Shoe - Men's Basketball 15 Collegiate Gold/White",
    "adidas Performance Men's X 15.2 CT Soccer Shoe,Green/Black/White,10.5 M US",
    "adidas Men's Adizero 8.0 SK Football Shoe, White/Power red/Active Red, 15 M US",
    "adidas Men's Energyfalcon Running Shoe, Grey Six/Glory Blue/core Black, 15 M US",
    "adidas Men's Freak Carbon Low Football Shoe, Black/Night Metallic/Grey, 15 M US",
    "adidas Men's ZG21 Golf Shoe",
    "adidas Men's Harden Stepback Basketball Shoe, core Black/Grey Six/FTWR White, 15 M US",
    "adidas Men's Athletic Cushioned Quarter Sock (6-Pair), White/Black, XL, (Shoe Size 12-15)",
    "adidas Men's FORGEFIBER BOA Golf Shoe dark marine/shock cyan/ftwr white 15 M/W US",
    "adidas Men's Athletic Cushioned Crew Socks (6-Pair), Black/Aluminum 2, XL, (Shoe Size 12-15)",
    "adidas Dame 5 Shoes Men's, Black, Size 7.5",
    "adidas Ultraboost S&L Shoes Men's, Black, Size 8.5",
    "adidas Ultraboost S&L Shoes Men's, White, Size 8.5",
    "adidas Ultraboost S&L Shoes Men's, Black, Size 8",
    "adidas Ultraboost Shoes Men's, Black, Size 7.5",
    "adidas Supernova Shoes Men's, Orange, Size 7.5",
    "adidas Men's Icon V Bounce Mid Baseball Shoe, FTWR White/Silver met./FTWR White, 15 M US"
]